In [ ]:
import sys
import numpy as np
import pandas as pd 

sys.path.append("../input/kaggle-chaii-2021-repo")

In [ ]:
!ls ../input/

In [ ]:
sub_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv")
sub_df["PredictionString"] = " "
sub_df

In [ ]:
tst_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
tst_df

In [ ]:
trn_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")
trn_df

In [ ]:
import Levenshtein

def jaccard(text_true: str, text_pred: str):
    a = set(text_true.lower().split())
    b = set(text_pred.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

from pandas import DataFrame

def _fill_sub_by_train_jaccard(sub_df: DataFrame, thresh: float) -> None:
    trn_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")
    tst_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
    # tst_df.loc[0, "question"] = trn_df.loc[0, "question"]
    if len(sub_df) != len(tst_df):
        raise Exception("len(sub_df) != len(tst_df)")
    tst_df = (
        tst_df.set_index("id")
        .loc[sub_df["id"].values]
        .reset_index()
        .reset_index(drop=True)
    )
    for i, tst_row in tst_df.iterrows():
        cur_i = -1
        cur_score = -1
        cur_answer_text = ""
        for _, trn_row in trn_df.iterrows():
            score = jaccard(
                text_true=tst_row["question"], text_pred=trn_row["question"]
            )
            if score > cur_score:
                cur_i = i
                cur_score = score
                cur_answer_text = trn_row["answer_text"]
        if cur_score >= thresh:
            print("COME HERE")
            sub_df.loc[cur_i, "PredictionString"] = cur_answer_text

def _fill_sub_by_train_leven(sub_df: DataFrame, thresh: float) -> None:
    trn_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/train.csv")
    tst_df = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
    # tst_df.loc[0, "question"] = trn_df.loc[0, "question"]
    if len(sub_df) != len(tst_df):
        raise Exception("len(sub_df) != len(tst_df)")
    tst_df = (
        tst_df.set_index("id")
        .loc[sub_df["id"].values]
        .reset_index()
        .reset_index(drop=True)
    )
    for i, tst_row in tst_df.iterrows():
        cur_i = 1_000_000_000
        cur_score = 1_000_000_000
        cur_answer_text = ""
        for _, trn_row in trn_df.iterrows():
            score = Levenshtein.distance(tst_row["question"], trn_row["question"])
            if score < cur_score:
                cur_i = i
                cur_score = score
                cur_answer_text = trn_row["answer_text"]
        if cur_score <= thresh:
            print("COME HERE 2")
            sub_df.loc[cur_i, "PredictionString"] = cur_answer_text

In [ ]:
_fill_sub_by_train_jaccard(sub_df=sub_df, thresh=0.8)
_fill_sub_by_train_leven(sub_df=sub_df, thresh=2)

In [ ]:
sub_df.to_csv("submission.csv", index=False)

In [ ]:
sub_df = pd.read_csv("submission.csv")
display(sub_df.shape, sub_df.head())